## Federated Learning in Blockchain for Data Privacy

In [11]:
import random
import namegenerator
import hashlib as hasher
import torch
import random
import time
# Creating list for names of a transactions
names = ["Transaction 1", "Transaction 2", "Transaction 3", "Transaction 4", "Transaction 5"]

# Creating list for tranasaction_id
transaction = [random.randint(10000, 99999) for i in range(len(names))]

# Labels for each each transaction either it was anamoly or correct
labels = ["Correct", "Anomaly", "Correct", "Anomaly", "Correct"]

# Printing the results
print("Names:", names)
print("Transaction ID:", transaction)
print("Labels:", labels)

def create_genesis_block():
  """Creates the genesis block."""
  timestamp = int(time.time())
  return Block(names[0], transaction[0], labels[0], timestamp)

def next_block(last_block, j):
  """Creates a new block."""
  timestamp = int(time.time())
  this_name = names[j]
  this_transaction_id = transaction[j]
  this_label = labels[j]
  this_hash = last_block.hash
  return Block(this_name, this_transaction_id, label, timestamp, this_hash)

class Block:
  """A block in the blockchain."""

  def __init__(self, name, transaction_id, label, timestamp, previous_hash=None):
    self.name = name
    self.transaction_id = transaction_id
    self.label = label
    self.timestamp = timestamp
    self.previous_hash = previous_hash
    self.hash = self.hash_block()

  def hash_block(self):
    """Computes the hash of the block."""
    sha = hasher.sha256()
    sha.update(str(self.name).encode('utf-8') +
               str(self.transaction_id).encode('utf-8') +
               str(self.label).encode('utf-8') +
               str(self.timestamp).encode('utf-8') +
               str(self.previous_hash).encode('utf-8'))
    return sha.hexdigest()

# Create the blockchain
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
num_of_blocks_to_add = len(names)

# Add blocks to the blockchain
for i in range(1, num_of_blocks_to_add):
  block_to_add = next_block(previous_block, i)
  blockchain.append(block_to_add)
  previous_block = block_to_add

# Print the blockchain
print("************************** All block details ****************************")
for block in blockchain:
  print(f"Name: {block.name}")
  print(f"Transaction ID: {block.transaction_id}")
  print(f"Label: {block.label}")
  print(f"Timestamp: {block.timestamp}")
  print(f"Hash: {block.hash}")
  print()

# Creating a dictionary to store the remote transactions
remote_transactions = {}

# Setting each transaction to remote
for i in range(len(names)):
  remote_transactions[names[i]] = {
    "transaction_id": transaction[i],
    "label": labels[i]
  }

# Printing the remote transactions
print("******************************** remote_transactions ********************************")
for key, value in remote_transactions.items():
  print(f"{key}: {value}")

# Creating a list of datasets
datasets = []

# Adding a check to make sure that the transaction ID and label are both present in the dataset
for i in range(len(names)):
  if names[i] in remote_transactions and transaction[i] in remote_transactions:
    datasets.append((names[i], remote_transactions[names[i]]))

# Printing the datasets
print("******************************** datasets values ***********************************")
for dataset in datasets:
  print(dataset)


Names: ['Transaction 1', 'Transaction 2', 'Transaction 3', 'Transaction 4', 'Transaction 5']
Transaction ID: [98942, 31938, 51234, 87906, 67813]
Labels: ['Correct', 'Anomaly', 'Correct', 'Anomaly', 'Correct']
************************** All block details ****************************
Name: Transaction 1
Transaction ID: 98942
Label: Correct
Timestamp: 1697861909
Hash: 9b95421c346be3a755a4a1a6ae3e715f8686be2f2bd6d3f7769f924c9fa78737

Name: Transaction 2
Transaction ID: 31938
Label: 1
Timestamp: 1697861909
Hash: 5bbacdc02636adba1b3933eb42c0f0249b6c682e91b146f5fddfff41a1426836

Name: Transaction 3
Transaction ID: 51234
Label: 1
Timestamp: 1697861909
Hash: 3c06602f3056578d0958070cfab905c7ea508992ab2d05af2e44cc1026aa4279

Name: Transaction 4
Transaction ID: 87906
Label: 1
Timestamp: 1697861909
Hash: b218ea0756b6a70ec677ac60fc9d2d12676928bdeb2f9c4de7badd521dfdb7e3

Name: Transaction 5
Transaction ID: 67813
Label: 1
Timestamp: 1697861909
Hash: 3de68e24e6d156e75238520c654addbbfa68f058446022cbe713

In [14]:
import torch
import random
import namegenerator
import hashlib as hasher
import time

from torch import nn
from torch import optim


In [20]:
def train(iterations = 20):
  model = nn.Linear(50,22)
  optimizer_fed = optim.SGD(params = model.parameters(), lr = 0.1)
  for iter in range(iterations):
    for data, target in datasets:
      model = model.send(data.location)
      optimizer_fed.zero_grad()
      pred = model(data)
      loss = (( pred - target) ** 2).sum()
      loss.backward()
      optimizer_fed.step()
      model = model.get()
      print(loss.get())
      #torch.save(model.state_dict(), "model.pt")

      


In [21]:
train()

In [23]:
import torch

model = nn.Linear(50, 22)

def train(iterations = 20):
  model = nn.Linear(50,22)
  optimizer_fed = optim.SGD(params = model.parameters(), lr = 0.1)
  for iter in range(iterations):
    for data, target in datasets:
      model = model.send(data.location)
      optimizer_fed.zero_grad()
      pred = model(data)
      loss = (( pred - target) ** 2).sum()
      loss.backward()
      optimizer_fed.step()
      model = model.get()
      print(loss.get())
      #torch.save(model.state_dict(), "model.pt")

# Save the trained model
torch.save(model.state_dict(), "model.pt")


In [24]:
model = nn.Linear(50, 22)
model.load_state_dict(torch.load("model.pt"))


<All keys matched successfully>

In [25]:
# Create a new dataset
new_data = torch.randn((1, 50))

# Make a prediction
prediction = model(new_data)

# Print the prediction
print(prediction)


tensor([[ 0.6978,  0.4367, -0.8118,  0.1862,  0.0659,  0.2834,  0.0105,  0.0184,
          0.0438, -1.1972, -0.1141,  0.3335,  0.0969, -0.3249,  0.7084, -0.2961,
          0.0245, -0.4710, -0.3482,  0.2798,  0.9593,  0.8677]],
       grad_fn=<AddmmBackward0>)
